In [ ]:
def create_dict(df, col):
    """takes a dataframe with a Participant column
    and splits it into a dictionary object containing a dataframe per participant
    in order for this to work the df columns has to have a participant column"""
    
    columns = list(df[col].unique()) 
    
    dict_df = {}
    for x in columns:
        dict_df[x] = df[df[col] == x]
    return(dict_df)

def dict_to_df(x, colname):
    x = pd.Series(x).to_frame()
    x = x.rename(index = str, columns = {0: colname})
    x = x.reset_index()
    x = x.rename(index = str, columns = {'index': 'Participant'})
    x.loc[:,'Participant'] = x.loc[:,'Participant'].astype(int)
    return x

def return_perc(x):
    perc = x[x.Evidence == 'Yes']['FixDur'] / x.FixDur.sum() * 100
    return float(perc)


def numbers_pp(df, col):
    """takes a dataframe and the name of a column with discrete variables and returns
    a dictionary with key value pairs being participant and a value of interest 
    
    df = df with columns
    col = column of interest (Note that this column should be included in df with columns).
    """
    perc_yes = {}
    
    value = create_dict(df)
    
    for k in value.keys():
        value[k].loc[:,col] = value[k].loc[:,col].astype(str)
        value[k] = value[k].drop(['Participant'], axis = 1)
        
        perc_yes[k] = return_perc2(pd.DataFrame(value[k].groupby([col]).sum()).reset_index(), col)
    return perc_yes

def return_perc2(x, col):
    perc = x[x[col] == 'Yes'].loc[:,'FixDur'] / x.FixDur.sum() * 100
    return float(perc)

def get_unique (df, col):
    new_df = {}
    
    dict_of_df = create_dict(df)
    
    for i in dict_of_df.keys():
        #dict_of_df[i] = dict_of_df[i].drop(['Participant'], axis = 1)
        new_df[i] = len(list(dict_of_df[i][col].unique())) 
        
    return new_df


def deal_with_imbalance(df,k):

    #df.Group = df.Group.map(dict(Novice = int(1), CSI = int(2))) #turn classes into 0 and 1 
    
    #Here we chose a knn strategy with k=3 in order to resample the minority class.
    smote = SMOTE(sampling_strategy = 'auto', k_neighbors = 3 ,random_state = 2018)
    X, y = smote.fit_resample(np.array(df.iloc[:,0:3]), np.array(df.Group)) # resample classes with a KNN

    #MAKE COLUMNS
    cols = ['ROI', 'FixDur', 'Pixel']
    cols2 = ['Group']

    #Concatenate the arrays into a new dataframe
    df = pd.concat([pd.DataFrame(X, columns = cols), pd.DataFrame(y, columns = cols2)],axis = 1  )
    df['Image'] = k
    df[['ROI','FixDur']] = df[['ROI','FixDur']].astype('int32') #= int(df['ROI'])
    
    
    return df